In [1]:
import pandas as pd
import numpy as np

# train, val, test split

In [2]:
train = pd.read_csv('./Bigcontest_data/train_scaled_1hot.csv')
test = pd.read_csv('./Bigcontest_data/test_scaled_1hot.csv')

In [3]:
X = train.drop(columns='is_applied')
y = train['is_applied']

In [4]:
X

,loan_limit,loan_rate,yearly_income,desired_amount,existing_loan_cnt,existing_loan_amt,age,credit_score_cut,company_month,personal_rehabilitation,...,houseown_type_RENT,houseown_type_SPOUSE,purpose_BUSINESS,purpose_BUYCAR,purpose_BUYHOUSE,purpose_ETC,purpose_HOUSEDEPOSIT,purpose_INVEST,purpose_LIVING,purpose_SWITCHLOAN
0,-0.117253,0.969545,-0.116105,-0.179011,2.878903,0.803544,1.331713,-1.293773,0.800113,0.082443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.522356,0.969545,0.625588,-0.135971,1.299762,0.751646,0.602979,-1.293773,-0.249237,0.082443,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.882448,1.878418,0.625588,-0.135971,1.299762,0.751646,0.602979,-1.293773,-0.249237,0.082443,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.567368,0.190510,0.064830,0.122271,-0.279380,0.677866,-0.021650,-0.186640,1.119480,0.082443,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.027230,0.813738,0.064830,0.122271,-0.279380,0.677866,-0.021650,-0.186640,1.119480,0.082443,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10270006,-0.632337,0.572923,0.016681,-0.135971,1.931418,0.752795,0.602979,-1.293773,-0.614228,0.082443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10270007,-0.632337,0.572923,0.016681,-0.135971,1.931418,0.752795,0.602979,-1.293773,-0.614228,0.082443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10270008,-0.632337,0.572923,0.016681,-0.135971,1.931418,0.752795,0.602979,-1.293773,-0.614228,0.082443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10270009,-0.632337,0.572923,0.016681,-0.135971,1.931418,0.752795,0.602979,-1.293773,-0.614228,0.082443,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=190000)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=190000)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svc_model = SVC()
svc_model.fit(X_train,y_train)

In [ ]:
import joblib
joblib.dump(svc_model, "svc_model.pkl") # DIFF
#...
# my_model_loaded = joblib.load("my_model.pkl")

In [ ]:
ROOT_DIR = './Bigcontest_data/'
SCORE_PATH = ROOT_DIR + "score/"

os.makedirs(SCORE_PATH, exist_ok = True)
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [ ]:
# accuracy, precision, recall, f1 score
def model_validation(model, data, label, mode, file_title):
    pred = model.predict(data)
    print('#### ' + mode + ' score ####')
    print(f'accuracy : {accuracy_score(label, pred)}, precision : {precision_score(label, pred)}, recall : {recall_score(label, pred)}, f1 score : {f1_score(label, pred)}')
    df = pd.DataFrame([], columns=['metric', 'score'])
    df.loc[0] = ['accuracy', accuracy_score(label, pred)]
    df.loc[1] = ['precision', precision_score(label, pred)]
    df.loc[2] = ['recall', recall_score(label, pred)]
    df.loc[3] = ['f1 score', f1_score(label, pred)]

    save_data(df, file_title, SCORE_PATH)
    return df


In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score

# ROC curve, auc score
# proba  clf.predict_proba(X)[:, 1]

def model_auc(model, data, label, mode):

    proba = model.predict_proba(data)[::, 1]

    print('#### ' + mode + ' ROC AUC score ####')
    print(roc_auc_score(label, proba))

    print('#### ' + mode + ' ROC curve plotting####')
    fpr, tpr, _ = metrics.roc_curve(label, proba)

    plt.figure(figsize=(15,15))
    plt.plot(fpr, tpr)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.title(mode + " ROC curve")
    save_fig(mode + " ROC curve")
    plt.show()
    
